# <span style="color:#d3d1df">Ms Thesis Wage Data Preperation</span>

## <span style="color:#f1c232">Overview</span>

**To-Do's:** <br>

 1. Import datasets
 2. Concat different datasets(years)
 3. Delete Unnecessary observations/variables
 4. Do necessary variable transformations
 5. Delete Unnecessary observations/variables
 6. Export function


## <span style="color:#f1c232">Environment</span>

In [79]:
#Packages

import pandas as pd
import numpy as np
from Utilities import wages 
import eurostat

#Variables

#EARN_SES_14=["EARN_SES_AGT14","EARN_SES06_14","EARN_SES10_14","EARN_SES14_14","EARN_SES18_14"]

EARN_SES_47=["EARN_SES06_47","EARN_SES10_47","EARN_SES14_47","EARN_SES18_47"]

#EARN_SES_16=["EARN_SES_AGT16","EARN_SES06_16","EARN_SES10_16","EARN_SES14_16","EARN_SES18_16"]


## <span style="color:#f1c232">Data Preperation</span>

In [102]:
#Get the datasets via Eurostat API and concat them to get the inititial data 
df=pd.concat([eurostat.get_data_df(j, flags=True).assign(year=2006+4*i) for i, j in enumerate(EARN_SES_47)],ignore_index=True)

In [103]:
#Gender, earning type, and company size will not be examined in the research.  
df=df[(df['sex']=='T')&(df['indic_se']=='ERN')&(df['sizeclas']=='TOTAL')]
df=df.drop(['freq','sex','indic_se','sizeclas'], axis=1)

#Rename Country code column
df=df.rename(columns={'geo\TIME_PERIOD':'Code'})

#df=df[(df['geo\TIME_PERIOD'].isin(['AT','BE','BG','CH','CY','CZ','DE','DK','EE','EL','ES','FI','FR','HU','IE','IS','IT','LT','LU','LV','MT','NL','NO','PL','PT','RO','SE','SI','SK','TR',
#'UK']))]
df


,unit,nace_r1,isco88,Code,2006_value,2006_flag,year,currency,isco08,nace_r2,2010_value,2010_flag,2014_value,2014_flag,2018_value,2018_flag
1151,EUR,C,ISCO0,NO,NaN,: c,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1193,EUR,C,ISCO1,BG,2.44,,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1194,EUR,C,ISCO1,CY,23.55,,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1195,EUR,C,ISCO1,CZ,11.43,,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1196,EUR,C,ISCO1,DE,32.44,,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080852,PPS,NaN,NaN,NO,NaN,NaN,2018,NaN,TOTAL,S,NaN,NaN,NaN,NaN,20.95,
1080853,PPS,NaN,NaN,PL,NaN,NaN,2018,NaN,TOTAL,S,NaN,NaN,NaN,NaN,7.71,
1080854,PPS,NaN,NaN,SI,NaN,NaN,2018,NaN,TOTAL,S,NaN,NaN,NaN,NaN,8.51,
1080855,PPS,NaN,NaN,SK,NaN,NaN,2018,NaN,TOTAL,S,NaN,NaN,NaN,NaN,6.14,


In [104]:
#Merging unit and currency colums, eleminating unnessesary currencies
df['unit']=df['unit'].replace(np.nan, '', regex=True)+df['currency'].replace(np.nan, '', regex=True)
df=df.drop(['currency'], axis=1)
df=df[(df['unit']!= 'NAC')]
df['unit'].unique()

array(['EUR', 'PPS'], dtype=object)

In [105]:
#Arranging value and flag columns
df['value']=df['2006_value'].replace(np.nan, 0, regex=True)+df['2010_value'].replace(np.nan, 0, regex=True)+df['2014_value'].replace(np.nan, 0, regex=True)+df['2018_value'].replace(np.nan, 0, regex=True)
df=df.drop(['2006_value','2010_value','2014_value','2018_value'], axis=1)
df['flag']=df['2006_flag'].replace(np.nan, '', regex=True)+df['2010_flag'].replace(np.nan, '', regex=True)+df['2014_flag'].replace(np.nan, '', regex=True)+df['2018_flag'].replace(np.nan, '', regex=True)
df=df.drop(['2006_flag','2010_flag','2014_flag','2018_flag'], axis=1)

In [106]:
occupation=pd.DataFrame({
    'isco88': ['ISCO0', 'ISCO1', 'ISCO1-5', 'ISCO2', 'ISCO3', 'ISCO4', 'ISCO5', 'ISCO6', 'OC6-8', 'ISCO7', 'ISCO7-9', 'ISCO8', 'ISCO9','TOTAL'],
    'isco08': ['OC0', 'OC1', 'OC1-5', 'OC2', 'OC3', 'OC4', 'OC5', 'OC6', 'OC6-8', 'OC7', 'OC7-9', 'OC8', 'OC9','TOTAL'],
    'Description': ['Armed forces occupations', 'Managers', 'Non manual workers', 'Professionals', 'Technicians and associate professionals', 'Clerical support workers', 'Service and sales workers', 'Skilled agricultural, forestry and fishery workers', 'Skilled manual workers', 'Craft and related trades workers', 'Manual workers', 'Plant and machine operators and assemblers', 'Elementary occupations','Total']
            })
df['isco88']=df['isco88'].map(occupation.set_index('isco88')['isco08'])
df['isco08']=df['isco88'].replace(np.nan, '', regex=True)+df['isco08'].replace(np.nan, '', regex=True)

In [107]:
df['isco88'].unique()


array(['OC0', 'OC1', 'OC1-5', 'OC2', 'OC3', 'OC4', 'OC5', 'OC6', 'OC7',
       'OC7-9', 'OC8', 'OC9', 'TOTAL', nan], dtype=object)

In [108]:
df=df.drop(['isco88'], axis=1)

In [110]:
df

,unit,nace_r1,Code,year,isco08,nace_r2,value,flag
1151,EUR,C,NO,2006,OC0,NaN,0.00,: c
1193,EUR,C,BG,2006,OC1,NaN,2.44,
1194,EUR,C,CY,2006,OC1,NaN,23.55,
1195,EUR,C,CZ,2006,OC1,NaN,11.43,
1196,EUR,C,DE,2006,OC1,NaN,32.44,
...,...,...,...,...,...,...,...,...
1080852,PPS,NaN,NO,2018,TOTAL,S,20.95,
1080853,PPS,NaN,PL,2018,TOTAL,S,7.71,
1080854,PPS,NaN,SI,2018,TOTAL,S,8.51,
1080855,PPS,NaN,SK,2018,TOTAL,S,6.14,


In [113]:
#Define sector dataframe containing Nace Mapping and descriptions
S = [['A', 'A', 'Agriculture, forestry and fishing'],['C', 'B', 'Mining and quarrying'],['D', 'C', 'Manufacturing'],['E', 'D', 'Electricity, gas, steam and air conditioning supply'],['E', 'E', 'Water supply, sewerage, waste management and remediation activities'],['F', 'F', 'Construction'],['G', 'G', 'Wholesale and retail trade; repair of motor vehicles and motorcycles'],['H', 'I', 'Accommodation and food service activities'],['I', 'H', 'Transportation and storage'],['I', 'J', 'Information and communication'],['J', 'K', 'Financial and insurance activities'],['K', 'L', 'Real estate activities'],['K', 'M', 'Professional, scientific and technical activities'],['K', 'N', 'Administrative and support service activities'],['L', 'O', 'Public administration and defence; compulsory social security'],['M', 'P', 'Education'],['N', 'Q', 'Human health and social work activities'],['O', 'R', 'Arts, entertainment and recreation'],['O', 'S', 'Other service activities'],['P', 'T', 'Activities of households as employers; undifferentiated goods- and services-producing activities of households for own use'],['Q', 'U', 'Activities of extraterritorial organisations and bodies']]
sector = pd.DataFrame(S, columns=['nace_r1', 'nace_r2', 'description'])

In [116]:
df['deneme']=df['nace_r1'].map({item[0]: item[1] for item in S})
df['nace_r2']=df['deneme'].replace(np.nan, '', regex=True)+df['nace_r2'].replace(np.nan, '', regex=True)



In [120]:
df=df.drop(['nace_r1','deneme'], axis=1)
df

,unit,Code,year,isco08,nace_r2,value,flag
1151,EUR,NO,2006,OC0,BC,0.00,: c
1193,EUR,BG,2006,OC1,BC,2.44,
1194,EUR,CY,2006,OC1,BC,23.55,
1195,EUR,CZ,2006,OC1,BC,11.43,
1196,EUR,DE,2006,OC1,BC,32.44,
...,...,...,...,...,...,...,...
1080852,PPS,NO,2018,TOTAL,S,20.95,
1080853,PPS,PL,2018,TOTAL,S,7.71,
1080854,PPS,SI,2018,TOTAL,S,8.51,
1080855,PPS,SK,2018,TOTAL,S,6.14,


In [121]:
df['nace_r2'].unique()

array(['BC', 'C-E', 'C-F', 'C-K', 'C-O', 'C-O_X_L', 'CD', 'EE', 'FF',
       'GG', 'G-I', 'G-K', 'IH', 'JI', 'KJ', 'J_K', 'NK', 'OL', 'PM',
       'M-O', 'QN', 'SO', 'B-F', 'B-N', 'B-S', 'B-S_X_O', 'F', 'G', 'G-J',
       'G-S', 'G-S_X_O', 'H', 'O', 'O-S', 'P', 'P-S', 'S', 'B', 'B-E',
       'C', 'D', 'E', 'I', 'J', 'K', 'K-N', 'L', 'M', 'N', 'Q', 'R'],
      dtype=object)

In [ ]:
df[(df['Code']=='NL')&(df['isco08']=='OC4')]['deneme'].unique()

array(['B', nan, 'C', 'E', 'F', 'G', 'I', 'J', 'K', 'N', 'O', 'P', 'Q',
       'S'], dtype=object)

In [118]:
df['nace_r2'].unique()

array(['BC', 'C-E', 'C-F', 'C-K', 'C-O', 'C-O_X_L', 'CD', 'EE', 'FF',
       'GG', 'G-I', 'G-K', 'IH', 'JI', 'KJ', 'J_K', 'NK', 'OL', 'PM',
       'M-O', 'QN', 'SO', 'B-F', 'B-N', 'B-S', 'B-S_X_O', 'F', 'G', 'G-J',
       'G-S', 'G-S_X_O', 'H', 'O', 'O-S', 'P', 'P-S', 'S', 'B', 'B-E',
       'C', 'D', 'E', 'I', 'J', 'K', 'K-N', 'L', 'M', 'N', 'Q', 'R'],
      dtype=object)

In [112]:
df

,unit,nace_r1,Code,year,isco08,nace_r2,value,flag
1151,EUR,C,NO,2006,OC0,NaN,0.00,: c
1193,EUR,C,BG,2006,OC1,NaN,2.44,
1194,EUR,C,CY,2006,OC1,NaN,23.55,
1195,EUR,C,CZ,2006,OC1,NaN,11.43,
1196,EUR,C,DE,2006,OC1,NaN,32.44,
...,...,...,...,...,...,...,...,...
1080852,PPS,NaN,NO,2018,TOTAL,S,20.95,
1080853,PPS,NaN,PL,2018,TOTAL,S,7.71,
1080854,PPS,NaN,SI,2018,TOTAL,S,8.51,
1080855,PPS,NaN,SK,2018,TOTAL,S,6.14,
